# Corrección atmosférica en bucle

A continuación se detalla la modificación del código de Sam Murphy realizada para corregir en bucle todas las imágenes de una colección.

## Incluir paquetes necesarios e iniciar GEE

In [1]:
# Uncomment the following line if condacolab is not installed
!pip install -q condacolab

**Importante**: Cuando se termina de ejecutar `condacolab.install()` el kernel se reinicia. Deberá volverse a ecutar el código y devolverá el mensaje "Everything looks OK!".

In [1]:
# Load conda commands
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# Download Py6S module and 6S compiled model
!conda install conda-forge::py6s

In [4]:
#@title Load Atmospheric (funciones atmospheric.py)
"""
atmospheric.py, Sam Murphy (2016-10-26)

Atmospheric water vapour, ozone and AOT from GEE

Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)

"""

# El modulo ee sera importado mas adelante
# import ee

class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

    # start of NEXT month
    m2 = m1.advance(1,'month')

    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()

    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.

    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)
    """

    # Point geometry required
    centroid = geom.centroid()

    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)

    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

    # water image
    water_img = ee.Image(water_ic.first())

    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)

    return water_Py6S_units



  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset

    OR

    uses our fill value (which is mean value for that latlon and day-of-year)

    """

    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):

      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

      # ozone image
      ozone_img = ee.Image(ozone_ic.first())

      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))

      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)

      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

      """

      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))

      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)

    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')

    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )

    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.

    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """

    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])


    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )

      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))

      return img


    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)

    return AOT

In [3]:
# Load mandatory modules
import ee
from Py6S import *
import datetime
import math
import os
import sys

Iniciar la API de EarthEngine.

In [5]:
ee.Authenticate()

Inicializar un proyecto de Google Cloud en el que se encuentre activada la API de Earth Engine. El proyecto debe estar vinculado con la cuenta de google para la que se ha realizado el registro con `ee.Authenticate()`.

In [6]:
ee.Initialize(project='id') # Project ID

Definir la función que aplica la constante para obtener valores de reflectividad a TOA en las imágenes de la colección COPERNICUS/S2_HARMONIZED. Permite conservar las propiedades de la imagen original.

In [17]:
def reflectance(img):
  """
  Convert to reflectance values and mantain original image properties.

  The 'system:index' property is written inside fileID key due to:
  - It's not added in the exported image
  - This property is mandatory to perform JOIN operation with
  COPERNICUS/S2_CLOUD_PROBABILITY image collection.
  """
  r = img.divide(10000)
  # Set new property
  r = r.set('fileID', img.id());
  # Get property names in the original image
  props = img.propertyNames();
  # Return the new image with the original image properties
  return r.copyProperties(img, props);


## Se definen las tres funciones creadas por Sam Murphy

In [8]:
def spectralResponseFunction(bandname):

  """
  Extract spectral response from predefined Sentinel2 bands.
  Use PredefinedWavelengths class in Py6s module.
  """
  bandSelect = {
    'B1':PredefinedWavelengths.S2A_MSI_01,
    'B2':PredefinedWavelengths.S2A_MSI_02,
    'B3':PredefinedWavelengths.S2A_MSI_03,
    'B4':PredefinedWavelengths.S2A_MSI_04,
    'B5':PredefinedWavelengths.S2A_MSI_05,
    'B6':PredefinedWavelengths.S2A_MSI_06,
    'B7':PredefinedWavelengths.S2A_MSI_07,
    'B8':PredefinedWavelengths.S2A_MSI_08,
    'B8A':PredefinedWavelengths.S2A_MSI_8A,
    'B9':PredefinedWavelengths.S2A_MSI_09,
    'B10':PredefinedWavelengths.S2A_MSI_10,
    'B11':PredefinedWavelengths.S2A_MSI_11,
    'B12':PredefinedWavelengths.S2A_MSI_12
  }
  return Wavelength(bandSelect[bandname])

def toa_to_rad(toa, bandname, info, solar_z, scene_date):
  """
  Convert TOA reflectance to Radiance units (values to perform 6S correction)
  """
  # Get metadata from S2 image
  ESUN = info['SOLAR_IRRADIANCE_'+bandname]
  solar_angle_correction = math.cos(math.radians(solar_z))
  # Compute Earth-Sun distance (doy)
  doy = scene_date.timetuple().tm_yday
  d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))
  # http://physics.stackexchange.com/
  # questions/177949/earth-sun-distance-on-a-given-day-of-the-year

  # Conversion factor
  multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
  # Apply radiance equation
  rad = toa.select(bandname).multiply(multiplier)
  return rad

def surface_reflectance(s, toa, bandname, info, solar_z, scene_date):
  """
  Compute BOA reflectance (wavelenght related) with 6S outputs.
  """
  # Include to s object the current band spectral response
  s.wavelength = spectralResponseFunction(bandname)

  # Execute 6S model
  s.run()

  # Extract atmospheric terms
  Edir = s.outputs.direct_solar_irradiance # Direct solar irradiance
  Edif = s.outputs.diffuse_solar_irradiance # Diffuse solar irradiance
  Lp = s.outputs.atmospheric_intrinsic_radiance # Path radiance
  absorb = s.outputs.trans['global_gas'].upward # Absorption transmissivity
  scatter = s.outputs.trans['total_scattering'].upward # Scattering transmissivity
  tau2 = absorb*scatter # Total transmissivity

  # Get radiance values
  rad = toa_to_rad(toa, bandname, info, solar_z, scene_date)
  # Compute the equation to get BOA reflectance
  ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))

  return ref

## Definir el AOI sobre el que corregir la imagen

In [9]:
# Study area from which:
# - Image collection is filtered
# - Region to export the corrected image
# - Atmospheric parameters are extracted
geom = ee.Geometry.Polygon([
    [-0.9570796519011493,40.98197275411647],
    [-0.5670650034636493,40.98197275411647],
    [-0.5670650034636493,41.45919658393617],
    [-0.9570796519011493,41.45919658393617],
    [-0.9570796519011493,40.98197275411647]])

# Another option to create a geom study area
# geom = ee.Geometry.Rectangle(-0.996, 41.508, -0.568, 40.992)

# Get BBOX coordinates (used to export the corrected image inside its AOI)
region = geom.bounds().getInfo()['coordinates']

## Creacion de la función para corregir imágenes en bucle

In [20]:
def conversion(img, asset_id):
  """
  Atmospherically correction of all images in an Image Collection.
  """
  # Save image adquisition date
  date = img.date()

  # Compute TOA reflectance
  toa = ee.Image(reflectance(img))

  # Write image metadata
  info = img.getInfo()['properties']

  # Scene date: Python uses seconds, EE miliseconds
  miliseconds = info['system:time_start']/1000
  scene_date = datetime.datetime.utcfromtimestamp(miliseconds)

  # Solar zenith angle
  solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

  # Atmospheric components
  # (functions created by Sam Murphy and load in an above chunk)
  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  # Atmospheric Optical Thickness
  aot = Atmospheric.aerosol(geom,date).getInfo()

  # Elevation from Shuttle Radar Topography mission (STRM) collection
  SRTM = ee.Image('CGIAR/SRTM90_V4')

  # Get elevation of the AOI center
  alt = SRTM.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': geom.centroid()}).get('elevation').getInfo()

  # Transform to kilometers, units used in Py6s
  km = alt/1000

  # 6S backbone
  s = SixS()
  # Load atmospheric components
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Continental
  s.aot550 = aot
  # Geometry Earth-Sun-Satellite
  s.geometry = Geometry.User()
  s.geometry.view_z = 0 # NADIR
  s.geometry.solar_z = solar_z # Solar zenith angle
  s.geometry.month = scene_date.month # month to compute Earth-Sun distance
  s.geometry.day = scene_date.day # Day to compute Earth-Sun distance
  s.altitudes.set_sensor_satellite_level()
  s.altitudes.set_target_custom_altitude(km) # Surface altitude

  # Apply conversion over each image band
  output = img.select('QA60')
  # Wavelenght dependent
  for band in ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']:
    # 6S Atmospheric Correction
    boa = surface_reflectance(s, toa, band, info, solar_z, scene_date)
    # Add correction band inside the output image
    output = output.addBands(boa)

  # Export corrected image
  # 1. Write additional custom properties
  dateString = scene_date.strftime("%Y-%m-%d")

  ref = output.set({
    'satellite':'Sentinel 2',
    'date':dateString,
    'aerosol_optical_thickness':aot,
    'water_vapour':h2o,
    'ozone':o3
  })
  # Add new properties to the image original ones
  props = toa.propertyNames()
  ref = ref.copyProperties(toa, props)

  # Image name (inside GEE Asset path)
  image_id = asset_id + 'S2SR_' + dateString

  # 2. Export options
  params = {
    'image': ee.Image(ref),
    'description': 'sentinel2_atmcorr_export',
    'assetId': image_id,
    'region': region,
    'crs': 'EPSG:4326',
    'scale': 20
  }
  # Create export task
  export = ee.batch.Export.image.toAsset(**params)

  # 3. Initialize task
  export.start()
  return print("Imagen " + image_id + " en proceso de descarga.")

##

## Aplicar la funcion de conversion en bucle a una coleccion GEE

Primero se define la colección y despues se aplica la conversión dentro de un bucle a todas las imágenes que contiene.

In [21]:
# Definir coleccion GEE
S2 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
  .filterBounds(geom)
  .filterDate('2015-10-01','2017-04-30')
  .filterMetadata('MGRS_TILE', 'equals', '30TXL')
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)
  .sort('system:time_start')
  .distinct('system:time_start'))

# Definir en una lista las imagenes a filtrar
features = S2.getInfo()['features']

# Definir la carpeta de destino (dentro de GEE)
assetID = 'users/iranzocristian/6s_test_2023/'

"""
CORRECCION DE LA COLECCION AUTOMATICAMENTE (bucle for)
1. Recorre cada imagen de la lista anterior
2. Obtiene su id
3. Se llama a la imagen de la coleccion GEE con el id anterior
4. Se aplica la funcion de conversion principal
"""
for i in features:
  id = i['id']
  conversion(ee.Image(id), assetID)
# Final del Script

Imagen users/iranzocristian/6s_test_2023/S2SR_2015-11-13 en proceso de descarga.


## Cargar la coleccion de imagenes corregidas

En la API de JavaScript (code editor) debería incluirse el siguiente código:

```javascript
var assetList = ee.data.listAssets("users/iranzocristian/6s_test_2023")['assets']
                    .map(function(d) { return d.name })
var collection = ee.ImageCollection(assetList)
```

In [ ]:
assetList = ee.data.listAssets("users/iranzocristian/6s_test_2023")['assets']
assetNames = [i['name'] for i in assetList]
collection = ee.ImageCollection(assetNames)
print('Has cargado una colección con ' + str(collection.size().getInfo()) + ' imgs corregidas.')